In [2]:
pip install geopy 

     |████████████████████████████████| 119 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 6.0 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install Nominatim

  Created wheel for Nominatim: filename=nominatim-0.1-py3-none-any.whl size=2363 sha256=9e887a11ae37e92ff9de926183cf1240199cfe18437fcf442cd932412f4a863e
  Stored in directory: /Users/ajdomingo/Library/Caches/pip/wheels/37/00/9e/d904c390bfb174830ad3dcfd62af5544cee7d88bed4f8acedd
Successfully built Nominatim
Note: you may need to restart the kernel to use updated packages.


In [2]:
from geopy.geocoders import Nominatim
import pandas as pd

In [ ]:
file = "Resources/Seattle_Housing_Cost_Burden.csv"
file_df = pd.read_csv(file)
file_df.head()